<a href="https://colab.research.google.com/github/jedrexd/WTUM_11_2022/blob/main/Projekt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [ ]:
!git clone https://github.com/jedrexd/WTUM_11_2022

In [18]:
df = pd.read_csv("/content/WTUM_11_2022/train.csv", header=0)
df = df.iloc[:,1:]



In [19]:
df.describe()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
count,1460.000000,1201.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1452.000000,1460.000000,1460.000000,...,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000,1460.000000
mean,56.897260,70.049958,10516.828082,6.099315,5.575342,1971.267808,1984.865753,103.685262,443.639726,46.549315,...,94.244521,46.660274,21.954110,3.409589,15.060959,2.758904,43.489041,6.321918,2007.815753,180921.195890
std,42.300571,24.284752,9981.264932,1.382997,1.112799,30.202904,20.645407,181.066207,456.098091,161.319273,...,125.338794,66.256028,61.119149,29.317331,55.757415,40.177307,496.123024,2.703626,1.328095,79442.502883
min,20.000000,21.000000,1300.000000,1.000000,1.000000,1872.000000,1950.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2006.000000,34900.000000
25%,20.000000,59.000000,7553.500000,5.000000,5.000000,1954.000000,1967.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,2007.000000,129975.000000
50%,50.000000,69.000000,9478.500000,6.000000,5.000000,1973.000000,1994.000000,0.000000,383.500000,0.000000,...,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,2008.000000,163000.000000
75%,70.000000,80.000000,11601.500000,7.000000,6.000000,2000.000000,2004.000000,166.000000,712.250000,0.000000,...,168.000000,68.000000,0.000000,0.000000,0.000000,0.000000,0.000000,8.000000,2009.000000,214000.000000
max,190.000000,313.000000,215245.000000,10.000000,9.000000,2010.000000,2010.000000,1600.000000,5644.000000,1474.000000,...,857.000000,547.000000,552.000000,508.000000,480.000000,738.000000,15500.000000,12.000000,2010.000000,755000.000000


In [80]:
df['MSSubClass'] = LabelEncoder().fit_transform(df['MSSubClass'].astype('str'))
df['MSZoning'] = LabelEncoder().fit_transform(df['MSZoning'].astype('str'))
df.replace('LotFrontage', {"NA": 0})
df['LotFrontage'] = StandardScaler().fit_transform(np.array(df['LotFrontage']).reshape(-1, 1));
df['LotArea'] = StandardScaler().fit_transform(df['LotArea'].values.reshape(-1, 1))
df['Street'] = LabelEncoder().fit_transform(df['Street'].astype('str'))
df['Alley'] = LabelEncoder().fit_transform(df['Alley'].astype('str'))
lotShapeDict = {"Reg": 1, "IR1": 2, "IR2": 3, "IR3": 4}
df.replace('LotShape', lotShapeDict)
df['LandContour'] = LabelEncoder().fit_transform(df['LandContour'].astype('str'))
df['LotConfig'] = LabelEncoder().fit_transform(df['LotConfig'].astype('str'))
df['LandSlope'] = LabelEncoder().fit_transform(df['LandContour'].astype('str'))
df['Neighborhood'] = LabelEncoder().fit_transform(df['Neighborhood'].astype('str'))
conditionDict = {"Artery": 1, "Feedr": 2, "Norm": 3,	"RRNn": 4, "RRAn": 5, "PosN": 6, "PosA": 7, "RRNe": 8, "RRAe": 9}
df.replace('Condition1', conditionDict)
df.replace('Condition2', conditionDict)
utilitiesDict = {"AllPub": 3, "NoSewr": 2, "NoSeWa": 1, "ELO": 0}
df.replace('Utilities', utilitiesDict);
df['BldgType'] = LabelEncoder().fit_transform(df['BldgType'].astype('str'))
df['HouseStyle'] = LabelEncoder().fit_transform(df['HouseStyle'].astype('str'))

# YearBuilt, YearRemodAdd, OverallCond, OverallQual: TBD

0      -0.208034
1       0.409895
2      -0.084449
3      -0.414011
4       0.574676
          ...   
1455   -0.331620
1456    0.615871
1457   -0.166839
1458   -0.084449
1459    0.203918
Name: LotFrontage, Length: 1460, dtype: float64


0        9
1        4
2        9
3       10
4        9
        ..
1455     9
1456     4
1457    10
1458     4
1459     4
Name: MSSubClass, Length: 1460, dtype: int64